## 데이터 로드

In [ ]:
def get_column_data(path_list, div, col):
    col_data = []
    for path in path_list:
        h = h5py.File(path, 'r')
        col_data.append(h[div][col][:])
        h.close()
    return np.concatenate(col_data)

def get_dataframe(path_list, div):
    pids = get_column_data(path_list, div, col='pid')
    products = get_column_data(path_list, div, col='product')
    bradns = get_column_data(path_list, div, col='brand')
    makers = get_column_data(path_list, div, col='maker')
    
    models = get_column_data(path_list, div, col='model')
    prices = get_column_data(path_list, div, col='price')
    updttms = get_column_data(path_list, div, col='updttm')
    bcates = get_column_data(path_list, div, col='bcateid')
    mcates = get_column_data(path_list, div, col='mcateid')
    scates = get_column_data(path_list, div, col='scateid')
    dcates = get_column_data(path_list, div, col='dcateid')
    
    df = pd.DataFrame({'pid':pids, 'product':products, 'brand':brands,
                      'maker':makers, 'model':models, 'price':prices,
                      'updttm':updttms, 'bcateid':bcates, 'mcateid':mcates,
                      'scateid':scates, 'dcateid':dcates})
    
    df['pid'] = df['pid'].map(lambda x: x.decode('utf-8'))
    df['product'] = df['product'].map(lambda x: x.decode('utf-8'))
    df['brand'] = df['brand'].map(lambda x: x.decode('utf-8'))
    df['maker'] = df['maker'].map(lambda x: x.decode('utf-8'))
    df['model'] = df['model'].map(lambda x: x.decode('utf-8'))
    df['updttm'] = df['updttm'].map(lambda x: x.decode('utf-8'))
    return df

In [ ]:
train_df = get_dataframe(train_path_list, 'train')
dev_df = get_dataframe(dev_path_list, 'dev')
test_df = get_dataframe(test_path_list, 'test')

train_df.shape, dev_df.shape, test_df.shape

## 카테고리 매핑

In [ ]:
import json
cate_json = json.load(open(os.path.join(RAW_DATA_DIR, 'cate1.json')))

bid2nm = dict([(cid, name) for cid, name in cate_json['b'].items()])
mid2nm = dict([(cid, name) for cid, name in cate_json['m'].items()])
sid2nm = dict([(cid, name) for cid, name in cate_json['s'].items()])
did2nm = dict([(cid, name) for cid, name in cate_json['d'].items()])

# 데이터 프레임 안에 데이터 == 딕셔너리 키 ==> Value로 변환
train_df['bcatenm'] = train_df['bcateid'].map(bid2nm)
train_df['mcatenm'] = train_df['mcateid'].map(mid2nm)
train_df['scatenm'] = train_df['scateid'].map(sid2nm)
train_df['dcatenm'] = train_df['dcateid'].map(did2nm)

In [3]:
# dictionary의 안에 dictionary key value 받아오기
temp = {'ex':{'query':5, 'key':'vo', 'value':True}}
print(temp['ex'].items())
print([(a, b) for a, b in temp['ex'].items()])
dict([(a, b) for a, b in temp['ex'].items()])

dict_items([('query', 5), ('key', 'vo'), ('value', True)])
[('query', 5), ('key', 'vo'), ('value', True)]


{'query': 5, 'key': 'vo', 'value': True}

In [9]:
# 데이터의 특정 값을 dict 안에 있는 Value로 변환
import pandas as pd
tmp = dict([(a, b) for a, b in temp['ex'].items()])
pd.DataFrame([5, 'key', 'query'])[0].map(tmp)

0    NaN
1     vo
2      5
Name: 0, dtype: object

## Drop할 Col 선별

76p

In [ ]:
def get_vc_df(df, col):
    vc_df = df[col].value_counts().reset_index()
    vc_df.columns = [col, 'count'] # value_counts 를 df로 변환한 뒤, 이름 설정
    vc_df['percentage'] = (vc_df['count'] / vc_df['count'].sum())*100
    return vc_df

vc_df = get_vc_df(train_df, 'brand') # maker는 'maker' 만 추가하면 됨!
vc_df.head(10)
# brand, maker, model은 null 값이 많다. 즉, 예측 하는 데 정보가 담겨 있지 않은 경우가 많다
# 상품명에 이미 포함된 정보이다